In [6]:
import json

with open('data/data_file_example.json') as f:
    data = json.load(f)

In [10]:
data['records'][0]

{'datasetid': 'vlille-realtime',
 'recordid': '30dbe8287bbc3774c77c455b9fd901b1a8fc5f92',
 'fields': {'nbvelosdispo': 0,
  'nbplacesdispo': 0,
  'libelle': 136,
  'adresse': 'RUE CHAMBORD',
  'nom': 'RUE CHAMBORD',
  'etat': 'RÉFORMÉ',
  'commune': "VILLENEUVE D'ASCQ",
  'etatconnexion': 'DÉCONNECTÉ',
  'type': 'SANS TPE',
  'geo': [50.62972, 3.133873],
  'localisation': [50.62972, 3.133873],
  'datemiseajour': '2022-11-29T10:47:16+01:00'},
 'geometry': {'type': 'Point', 'coordinates': [3.133873, 50.62972]},
 'record_timestamp': '2023-08-24T21:28:01.077+02:00'}

In [1]:
import base64
from datetime import datetime
from google.cloud import storage
from google.cloud import bigquery
import requests
import json
import pytz
import os

In [2]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "key-vlille.json"
url = 'https://opendata.lillemetropole.fr/api/records/1.0/search/?dataset=vlille-realtime&q=&rows=300&timezone=Europe%2FParis'
paris_tz = pytz.timezone('Europe/Paris')
str_time_paris = datetime.now(paris_tz).strftime('%Y-%m-%d_%H:%M:%S')

In [3]:
bucket_name = 'vlille_data_json'
project_name = 'vlille-396911'

In [11]:
def get_json_data(url):
    # extract data from API
    response = requests.get(url)
    return response.json()

In [31]:
data = get_json_data(url)
data['records']

[{'datasetid': 'vlille-realtime',
  'recordid': '30dbe8287bbc3774c77c455b9fd901b1a8fc5f92',
  'fields': {'nbvelosdispo': 0,
   'nbplacesdispo': 0,
   'libelle': 136,
   'adresse': 'RUE CHAMBORD',
   'nom': 'RUE CHAMBORD',
   'etat': 'RÉFORMÉ',
   'commune': "VILLENEUVE D'ASCQ",
   'etatconnexion': 'DÉCONNECTÉ',
   'type': 'SANS TPE',
   'geo': [50.62972, 3.133873],
   'localisation': [50.62972, 3.133873],
   'datemiseajour': '2022-11-29T10:47:16+01:00'},
  'geometry': {'type': 'Point', 'coordinates': [3.133873, 50.62972]},
  'record_timestamp': '2023-09-19T22:40:05.273+02:00'},
 {'datasetid': 'vlille-realtime',
  'recordid': '109068ede961983697d353e26d9f598979bf4525',
  'fields': {'nbvelosdispo': 6,
   'nbplacesdispo': 17,
   'libelle': 55,
   'adresse': '3 PLACE JACQUES FEBVRIER',
   'nom': "PORTE D'ARRAS",
   'etat': 'EN SERVICE',
   'commune': 'LILLE',
   'etatconnexion': 'CONNECTÉ',
   'type': 'AVEC TPE',
   'geo': [50.617847, 3.061758],
   'localisation': [50.617847, 3.061758],
  

In [41]:
client = bigquery.Client(project=project_name)
dataset_id = 'vlille_dataset'
table_id = 'schema_from_sh'
table_ref = client.dataset(dataset_id).table(table_id)
table = client.get_table(table_ref)  # API call

In [42]:
data_to_insert = []
for record in data['records']:
    data_to_insert.append(record)
client.insert_rows(table, data_to_insert)

[]

In [43]:
# display rows of a table in BigQuery
query = "SELECT * FROM `vlille-396911.vlille_dataset.schema_from_sh` LIMIT 10"

query_job = client.query(query)

In [44]:
query_job.result()

In [45]:
for row in query_job:
    print(row)

Row(('vlille-realtime', '30dbe8287bbc3774c77c455b9fd901b1a8fc5f92', {'nbvelosdispo': 0, 'nbplacesdispo': 0, 'libelle': 136, 'adresse': 'RUE CHAMBORD', 'nom': 'RUE CHAMBORD', 'etat': 'RÉFORMÉ', 'commune': "VILLENEUVE D'ASCQ", 'etatconnexion': 'DÉCONNECTÉ', 'type': 'SANS TPE', 'geo': [50.62972, 3.133873], 'localisation': [50.62972, 3.133873], 'datemiseajour': datetime.datetime(2022, 11, 29, 9, 47, 16, tzinfo=datetime.timezone.utc)}, {'type': 'Point', 'coordinates': [3.133873, 50.62972]}, datetime.datetime(2023, 9, 19, 20, 40, 5, 273000, tzinfo=datetime.timezone.utc)), {'datasetid': 0, 'recordid': 1, 'fields': 2, 'geometry': 3, 'record_timestamp': 4})
Row(('vlille-realtime', '109068ede961983697d353e26d9f598979bf4525', {'nbvelosdispo': 6, 'nbplacesdispo': 17, 'libelle': 55, 'adresse': '3 PLACE JACQUES FEBVRIER', 'nom': "PORTE D'ARRAS", 'etat': 'EN SERVICE', 'commune': 'LILLE', 'etatconnexion': 'CONNECTÉ', 'type': 'AVEC TPE', 'geo': [50.617847, 3.061758], 'localisation': [50.617847, 3.06175

In [6]:
from google.cloud import bigquery
import os

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "key-vlille-gcp.json"

client = bigquery.Client()

# Create a 'stations' table
table_id = "vlille-gcp.vlille_gcp_dataset.stations"
schema = [
    bigquery.SchemaField("id", "INT64"),
    bigquery.SchemaField("nom", "STRING"),
    bigquery.SchemaField("adresse", "STRING"),
    bigquery.SchemaField("commune", "STRING"),
    bigquery.SchemaField("type", "STRING"),
    bigquery.SchemaField("latitude", "FLOAT64"),
    bigquery.SchemaField("longitude", "FLOAT64"),
]
table = bigquery.Table(table_id, schema=schema)
table = client.create_table(table)  # Make an API request.
print(
    "Created table {}.{}.{}".format(table.project, table.dataset_id, table.table_id)
)

Created table vlille-gcp.vlille_gcp_dataset.stations


In [7]:
# On alimente la table stations une fois avec une requête:
import requests

url = "https://opendata.lillemetropole.fr/api/records/1.0/search/?dataset=vlille-realtime&rows=300&facet=libelle&facet=nom&facet=commune&facet=etat&facet=type&facet=etatconnexion"
response = requests.get(url)
data = response.json()

rows_to_insert = []
for record in data["records"]:
    rows_to_insert.append(
        (
            record["fields"]["libelle"], # id = libelle
            record["fields"]["nom"],
            record["fields"]["adresse"],
            record["fields"]["commune"],
            record["fields"]["type"],
            record["fields"]["localisation"][0],
            record["fields"]["localisation"][1],
        )
    )

table_id = "vlille-gcp.vlille_gcp_dataset.stations"
try:
    table = client.get_table(table_id)
    client.insert_rows(table, rows_to_insert)
    print("Inserted rows.")
except Exception as e:
    print(e)

Inserted rows.
